In [1]:
# import libraries
import pandas as pd
import numpy as np

# set view parameters
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

In [2]:
##----------------------VAX SEQUENCE----------------------##
# import vaccination data
vax = pd.read_csv("vax by ZIP.csv")
vax = vax.rename(columns={'Zcta Num' : 'zip_code'}).astype('str')

# fix vax dtypes
vax.priority_neighborhood = vax.priority_neighborhood.astype(bool)
vax.at_least_1_dose = vax.at_least_1_dose.astype('int64')
vax.indicator = vax.indicator.astype(float)
vax.fully_vaccinated = vax.fully_vaccinated.astype('int64')
vax.partially_vaccinated = vax.partially_vaccinated.astype('int64')
vax.perc_fully = vax.perc_fully.astype('float')
vax.perc_partially = vax.perc_partially.astype('float')
vax.pop_denominator = vax.pop_denominator.astype('int64')
vax.pop_estimate = vax.pop_estimate.astype('int64')

# determine citywide rate
citywide_rate = 303709 / vax.pop_estimate.sum() * 100
citywide_rate

4.578966206512041

In [3]:
##----------------------POVERTY SEQUENCE----------------------##

# import poverty data for all NYC ZIP codes
url = 'https://api.census.gov/data/2019/acs/acs5?get=B19013_001E,B17001_001E,B17001_002E&for=zip%20code%20tabulation%20area:10000,10001,10002,10003,10004,10005,10006,10007,10009,10010,10011,10012,10013,10014,10016,10017,10018,10019,10020,10021,10022,10023,10024,10025,10026,10027,10028,10029,10030,10031,10032,10033,10034,10035,10036,10037,10038,10039,10040,10044,10065,10069,10075,10103,10110,10111,10112,10115,10119,10128,10152,10153,10154,10162,10165,10167,10168,10169,10170,10171,10172,10173,10174,10177,10199,10271,10278,10279,10280,10282,10301,10302,10303,10304,10305,10306,10307,10308,10309,10310,10311,10312,10314,10451,10452,10453,10454,10455,10456,10457,10458,10459,10460,10461,10462,10463,10464,10465,10466,10467,10468,10469,10470,10471,10472,10473,10474,10475,11096,11101,11102,11103,11104,11105,11106,11109,11201,11203,11204,11205,11206,11207,11208,11004,11209,11210,11211,11212,11213,11214,11215,11216,11217,11218,11219,11220,11221,11222,11223,11224,11225,11226,11228,11229,11230,11231,11232,11233,11234,11235,11236,11237,11238,11239,11354,11355,11356,11357,11358,11359,11360,11361,11362,11363,11364,11365,11366,11367,11368,11369,11370,11371,11372,11373,11374,11375,11377,11378,11379,11385,11411,11412,11413,11414,11415,11416,11417,11418,11419,11420,11421,11422,11423,11424,11425,11426,11427,11428,11429,11430,11432,11433,11434,11435,11436,11451,11691,11692,11693,11694,11697&in=state:36'

# create pandas DataFrame from json, drop first row
poverty = pd.read_json(url)
poverty = poverty.drop(poverty.index[[0]])

#rename poverty columns
#0 = B19013_001E = household_median_income
#1 = B17001_001E = status_determined
#2 = B17001_002E = total_poverty
#3 = state
#4 = zip_code
poverty = poverty.rename(columns={0: 'household_median_income',
                                  1: 'status_determined',
                                  2: 'total_poverty',
                                  3: 'state',
                                  4: 'zip_code'})

# fix poverty dtypes
poverty.household_median_income = poverty.household_median_income.astype('int64')
poverty.status_determined = poverty.status_determined.astype('int64')
poverty.total_poverty = poverty.total_poverty.astype('int64')

#Filter no-population ZIP codes
poverty = poverty[poverty.status_determined > 0]

# determine poverty rate and create column
poverty_rate = poverty.total_poverty / poverty.status_determined * 100
poverty['poverty_rate'] = poverty_rate.round(2)

In [4]:
##----------------------INSURANCE SEQUENCE----------------------##
# import insurance data 
insurance = pd.read_csv("NY health insurance.csv").astype('str')
#insurance.population = insurance.population.replace('-','0')

# fix population dtype
insurance.population = insurance.population.astype('int64')
# remove non-populated ZIPs
insurance = insurance[insurance.population > 0]

# fix other dtypes
insurance.zip_code = insurance.zip_code.astype(str)
insurance.insured = insurance.insured.astype('int64')
insurance.percent_insured = insurance.percent_insured.astype(float)
insurance.insured = insurance.insured.astype('int64')
insurance.percent_uninsured = insurance.percent_uninsured.astype(float)

In [5]:
# merge insurance, poverty, and vax DataFrames
merged = pd.merge(insurance, poverty, on="zip_code")
merged = pd.merge(merged, vax, on="zip_code")
merged.sample()

,zip_code,population,insured,percent_insured,uninsured,percent_uninsured,household_median_income,status_determined,total_poverty,state,poverty_rate,priority_neighborhood,borough,neighborhoods,at_least_1_dose,geometry,indicator,fully_vaccinated,partially_vaccinated,perc_fully,perc_partially,pop_denominator,pop_estimate
87,11106,37659,35182,93.4,2477,6.6,68530,37550,4744,36,12.63,True,Queens,Astoria (South),3294,Polygon,4.48,1423,1871,4.48,5.89,37429,31753


In [6]:
# print them to a spreadsheet titled VAX_MASTER
merged.to_csv("VAX_MASTER.csv", index=False, encoding='utf-8')